In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from tcn import TCN
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Input, Model
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    batch_size, timesteps, input_dim = 1, 1, input_dim
    i = Input(batch_shape=(batch_size, timesteps, input_dim))
    o = TCN(return_sequences = False)(i)
    o = Dense(1)(o)

    model = Model(inputs=[i], outputs=[o])
    model.compile(optimizer = 'adam', loss = 'mse')

    x, y = scaled_X_train_data, scaled_y_train_data
    model.fit(x, y, epochs = 20)
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 1)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

time: 1.26 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/30_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/30_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/30_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/tcn/'+file[0:7]+'result_30min.csv')

Train on 2207 samples
Epoch 1/20
2207/2207 [==============================] - 7s 3ms/sample - loss: 0.3470
Epoch 2/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0235
Epoch 3/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0154
Epoch 4/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0108
Epoch 5/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0085
Epoch 6/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0074
Epoch 7/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0070
Epoch 8/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0081
Epoch 9/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0065
Epoch 10/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0062
Epoch 11/20
2207/2207 [==============================] - 6s 3ms/sample - loss: 0.0066
Epoch 12/20
2207/2207 [==================

2111/2111 [==============================] - 5s 3ms/sample - loss: 0.0039
Epoch 12/20
2111/2111 [==============================] - 6s 3ms/sample - loss: 0.0039
Epoch 13/20
2111/2111 [==============================] - 6s 3ms/sample - loss: 0.0038
Epoch 14/20
2111/2111 [==============================] - 6s 3ms/sample - loss: 0.0037
Epoch 15/20
2111/2111 [==============================] - 6s 3ms/sample - loss: 0.0038
Epoch 16/20
2111/2111 [==============================] - 5s 3ms/sample - loss: 0.0037
Epoch 17/20
2111/2111 [==============================] - 5s 3ms/sample - loss: 0.0037
Epoch 18/20
2111/2111 [==============================] - 5s 3ms/sample - loss: 0.0037
Epoch 19/20
2111/2111 [==============================] - 5s 3ms/sample - loss: 0.0042
Epoch 20/20
2111/2111 [==============================] - 5s 3ms/sample - loss: 0.0036
544-ws-training.csv - 80.67493965782927
544-ws-training.csv - 16.866281300882118
Train on 2184 samples
Epoch 1/20
2184/2184 [===========================

In [4]:
rmse_list

{'584-ws-training.csv': 30.52474103809666,
 '567-ws-training.csv': 24.72760288730244,
 '596-ws-training.csv': 18.493471137460382,
 '552-ws-training.csv': 21.193898211932687,
 '544-ws-training.csv': 80.67493965782927,
 '540-ws-training.csv': 32.61718696523571}

time: 2.57 ms


In [5]:
mae_list

{'584-ws-training.csv': 22.82649781883918,
 '567-ws-training.csv': 18.566747480788827,
 '596-ws-training.csv': 13.23142280578613,
 '552-ws-training.csv': 15.342387391875942,
 '544-ws-training.csv': 16.866281300882118,
 '540-ws-training.csv': 25.940476184752907}

time: 7.33 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 34.70530664964286
Standard Deviation - RMSE : 21.134414559236085
Mean - MAE : 18.795635497154183
Standard Deviation - MAE : 4.358072628231493
time: 6.88 ms
